This file provides the code that we would use to estimate demand using PyBLP with micro-moments. 

It assumes that we have a dataframe (df) with the following variables: 

'product_ids': unique product identifier
'Marca': brand 
'simplified_model': model without extra specifications (e.g. RAV4 4x4 premium and RAV4 GT would appear as RAV4)


'prices': price 

'Rut Unidad de Compra': unique buyer identifier 
'Region': geographic region of the buyer 
'Sector': type of buyer (e.g. hospital, university)
'Nro LicitaciÃ³n PÃºblica': FA to which it belongs (2017, 2021 or 2023)
'Cantidad': quantity of vehicles purchasd 
'Rut Proveedor': unique seller identifier 
'Nombre Proveedor Sucursal': name of the seller 
'x1', 'x2', 'x3', 'x4', 'x5', 'x6': product characteristics where 'x6' is the type of vehicle
'month'
'year'
'semester'
'market_ids': for example each 'Region'-'semester' could be a market 
'N_i': number of buyers in the market
'Zona': geographic zone (North, Central, South) 
'k': category of the buyer, for example combinations of 'Sector' and 'Zona' 
'model_id': unique model identifier 
'prices'


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pandasgui import show
import socket
import numpy as np
from unidecode import unidecode
import unicodedata
import re
from difflib import SequenceMatcher
import pyblp


Functions

In [2]:
def calculate_market_shares_with_outside_good(df_transac, outside_adjustment = 2, max = 0.4):
    """
    Calculate market shares using market-specific N_i from the DataFrame

    outside_adjustment: divides all the outside shares by that number to decrease them 
    max: puts ceiling on the outside shares
    """
    # Group by product (j) and market (t) to count purchases
    product_counts = df_transac.groupby(['product_ids', 'market_ids']).size().reset_index(name='purchases')
    
    # Get N_i for each market (taking first occurrence since it's constant within market)
    market_populations = df_transac.groupby('market_ids')['N_i'].first().reset_index()
  
    # Merge N_i with product counts
    product_counts = product_counts.merge(market_populations, on='market_ids')
    
    # Calculate market shares
    product_counts['shares'] = product_counts['purchases'] / product_counts['N_i']
    product_counts['shares_real'] =product_counts['purchases'] / product_counts['N_i']
                                               
    x_vars = [col for col in df_transac.columns if col.startswith('x')] #list of variables starting with x 
    
    # Get product characteristics
    product_chars = df_transac.groupby(['product_ids', 'market_ids']).agg(
        {var: 'first' for var in x_vars} | {'prices': 'first', 'model_id': 'first'}
    ).reset_index()
    
    # Merge market shares with product characteristics
    final_df = product_chars.merge(product_counts[['product_ids', 'market_ids', 'shares', 'shares_real', 'N_i']], on=['product_ids', 'market_ids'])
    
    # Calculate outside good share for each market
    market_shares = final_df.groupby('market_ids')['shares'].sum()
    outside_shares = 1 - market_shares
    outside_shares = outside_shares / outside_adjustment
    outside_shares = np.minimum(outside_shares, max)
    market_shares_real = final_df.groupby('market_ids')['shares_real'].sum()
    outside_shares_real = 1 - market_shares_real
    
    
    # Create outside good rows
    markets = df_transac['market_ids'].unique()
    outside_goods = pd.DataFrame()
    outside_goods['market_ids'] = markets
    outside_goods['product_ids'] = 0

    for var in x_vars:
        if pd.api.types.is_numeric_dtype(df_transac[var]):
            outside_goods[var] = 0  # Set numeric variables to 0
        else:
            outside_goods[var] = '-'  # Set categorical variables to '-'
    
    outside_goods['prices'] = 0
    outside_goods['shares'] = [outside_shares[t] for t in markets]
    outside_goods['shares_real'] = [outside_shares_real[t] for t in markets]

    # Combine outside goods with other products
    final_df = pd.concat([final_df, outside_goods])
    
    # Sort by market and product ID
    final_df = final_df.sort_values(['market_ids', 'product_ids'])
    
    return final_df


def create_competition_variables(df):
    """
    Function to create the 'demand_instruments'
    """
    
    return df

In [ ]:
product_data = calculate_market_shares_with_outside_good(df, outside_adjustment= 6, max = .3)

## Py-BLP with micro-moments. 

product_data is the data with the prices/shares/prod char,  in the documentation called product data
agent_data is the data with the demographics, in our case it has the distribution of demographics. 



For each market create one obs. for each local agency in the market. 

### Loading data

In [294]:
# Generate dummy variables
dummies = pd.get_dummies(product_data['x6'])

product_data.drop(columns=['x6'], inplace=True)

# Rename the columns of the dummies to sequential names (x6, x7, ...)
dummies.columns = [f'x{i}' for i in range(6, 6 + dummies.shape[1])]

# Concatenate the dummies with the original DataFrame
product_data = pd.concat([product_data, dummies], axis=1)


create product_data instrumetns 

In [16]:
product_data = create_competition_variables(product_data)

In [17]:
# Step 1: Map each market_id to its corresponding Region
market_region_df = df[['market_ids', 'Region']].drop_duplicates()

# Step 2: Count the number of observations for each region-sector pair
region_sector_counts = df.groupby(['Region', 'Sector', 'k']).size().reset_index(name='count')

# Step 3: Merge the counts into a combined dataset for each market_id and its region
agent_data = (
    market_region_df
    .merge(region_sector_counts, on='Region', how='left')
)

agent_data = agent_data.loc[agent_data.index.repeat(agent_data['count'])]
agent_data = agent_data.reset_index(drop=True)


unique_k = sorted(agent_data['k'].unique()) #list of different k values

dummy_vars = []
for i, k_value in enumerate(unique_k, 1):
    
    var_name = f'a{i}'
    agent_data[var_name] = (agent_data['k'] == k_value).astype(int)
    dummy_vars.append(var_name)
    
# Create the formula string
dummy_vars
agent_formulation_string =' + '.join(dummy_vars)
agent_data.drop(['Region', 'Sector'], axis=1, inplace=True)

In [18]:
agent_data['N'] = agent_data['market_ids'].value_counts()[agent_data['market_ids']].values
agent_data = agent_data.drop('count', axis=1)
agent_data['weights'] = 1 / agent_data['N']
m = 15 
for i in range(0, m ): #important: nodes have to start from 0
    agent_data[f'nodes{i}'] = np.random.chisquare(df=3, size=len(agent_data))

In [19]:
product_data

,product_ids,market_ids,x1,x2,x3,x4,x5,prices,model_id,shares,...,shares_sum,x6,x7,x8,x9,x10,demand_instruments_0,demand_instruments_1,demand_instruments_2,demand_instruments_3
15,0,0,-,0.0,0.0,-,-,0.0,NaN,0.544218,...,1.000000,True,False,False,False,False,0.000000e+00,0.000000,0.000000,0.000000
111,1537010,0,delantero - integral,5253.0,1857.0,climatizador bizona,antideslumbrante manual,23249664.0,44.0,0.068027,...,1.000000,False,False,False,False,True,1.697197e+07,0.800000,5356.000000,1881.000000
140,1537046,0,delantero - integral,5335.0,1815.0,climatizador bizona,antideslumbrante manual,14654600.0,6.0,0.068027,...,1.000000,False,True,False,False,False,1.869099e+07,0.800000,5339.600000,1889.400000
149,1537048,0,delantero - integral,5335.0,1815.0,climatizador bizona,antideslumbrante manual,18958060.0,6.0,0.068027,...,1.000000,False,True,False,False,False,1.783030e+07,0.800000,5339.600000,1889.400000
218,1538481,0,delantero - integral,5732.0,1923.0,climatizador trizona,antideslumbrante automÃ¡tico,16017995.0,14.0,0.068027,...,1.000000,False,True,False,False,False,1.841831e+07,0.800000,5260.200000,1867.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,2043071,216,delantero - integral,5335.0,1815.0,climatizador bizona,antideslumbrante manual,23958655.0,6.0,0.105374,...,1.305714,False,True,False,False,False,2.216525e+07,0.428571,5200.000000,1925.142857
2581,2044390,216,delantero - delantera,5253.0,1990.0,manual,antideslumbrante manual,27689076.0,154.0,0.105374,...,1.305714,False,False,True,False,False,2.163233e+07,0.571429,5211.714286,1900.142857
2646,2048605,216,delantero - delantera,5425.0,1882.0,manual,antideslumbrante manual,22162800.0,17.0,0.105374,...,1.305714,False,True,False,False,False,2.242180e+07,0.571429,5187.142857,1915.571429
2663,2049344,216,delantero - delantera,5546.0,2555.0,manual,antideslumbrante manual,24635700.0,174.0,0.105374,...,1.305714,False,False,False,False,False,2.206853e+07,0.571429,5169.857143,1819.428571


In [20]:
agent_data

,market_ids,k,a1,a2,a3,a4,a5,a6,a7,N,...,nodes5,nodes6,nodes7,nodes8,nodes9,nodes10,nodes11,nodes12,nodes13,nodes14
0,12,0,1,0,0,0,0,0,0,1200,...,9.144960,2.164913,2.742728,0.411056,5.377454,0.996362,2.323817,1.500411,0.408366,0.161630
1,12,0,1,0,0,0,0,0,0,1200,...,11.306754,19.871454,2.349071,8.416804,2.627442,1.070439,1.459562,0.847405,4.507280,2.360715
2,12,0,1,0,0,0,0,0,0,1200,...,1.224990,0.801937,2.037401,1.700438,1.467424,2.382328,2.148180,2.021020,2.954609,4.285607
3,12,0,1,0,0,0,0,0,0,1200,...,4.756356,5.110103,0.271105,2.540345,3.613013,4.144346,0.483197,2.711053,0.358984,5.419879
4,12,0,1,0,0,0,0,0,0,1200,...,0.418697,0.941283,2.191737,0.907711,3.321904,4.424199,3.021464,0.841830,6.190847,1.287114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52871,216,4,0,0,0,0,1,0,0,65,...,0.438364,0.454442,2.609159,2.893024,0.062987,8.300748,0.275669,2.703512,2.799947,0.557371
52872,216,4,0,0,0,0,1,0,0,65,...,2.817884,4.169082,0.631015,2.399733,1.303227,1.864653,1.316167,0.759356,0.719655,4.257006
52873,216,5,0,0,0,0,0,1,0,65,...,0.219610,5.233705,1.832999,2.033618,0.071981,1.125377,2.019622,4.193413,2.564793,3.023404
52874,216,6,0,0,0,0,0,0,1,65,...,0.618353,3.417940,0.468662,6.107124,2.067359,2.721539,1.480029,3.798709,4.823185,2.314899


### setting up the problem

observations 
1. absorbed cause multicollinearity issues in the vars, be careful when using it. 

In [259]:
prod_vars = list(dummies.columns) #dummies of product category 

formulation_vars = char_vars + prod_vars

formulation_string2 = '1 + ' + ' + '.join(formulation_vars) + ' + prices'

X1_formulation = pyblp.Formulation(formulation_string2) #absorb the categorical variables. 

X2_formulation = pyblp.Formulation('1 + ' + ' + '.join(prod_vars) + ' + prices') #X2: non-linear part does not allow absorb

product_formulations = (X1_formulation, X2_formulation ) 


product_formulations

(1 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + prices,
 1 + x6 + x7 + x8 + x9 + x10 + prices)

In [ ]:
agent_formulation = pyblp.Formulation('1 +' + agent_formulation_string)
agent_formulation

problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data, costs_type='log')
problem


### Setting up micro moments

Create the observed moments composed by 
1. $$  G^2(\theta) = Pr(\{\text{i purchases }c\}| i\in k), \ c\in \{\text{SUV, Pick-up}\}  $$ 
2. $$     G^3(\theta) = \mathbb{E}[p_{jt}| i\in k]
$$

In [261]:
micro_dataset = pyblp.MicroDataset(
    name="transactions",
    observations= len(agent_data),
    compute_weights=lambda t, p, a: np.ones((a.size, 1 + p.size)), #define a function that computes the weights and will be used later 
)
micro_dataset

transactions: 52876 Observations in All Markets

In [262]:
## 1. create the first set of micro moments (G2) 
avg_outside_share = product_data[product_data['product_ids'] == 0]['shares'].mean() #average share of outside good

micro_statistics = df.groupby(['k', 'x6']).size().reset_index(name='count') #count of each k, x6 combination where x6 is the product category
micro_statistics['N'] = df['k'].value_counts()[micro_statistics['k']].values
micro_statistics['share'] = micro_statistics['count'] / micro_statistics['N']
micro_statistics['share'] = micro_statistics['share']/(1 + avg_outside_share) #normalize by the outside share

for i, k_value in enumerate(unique_k, 1):    
    micro_statistics[f'a{i}'] = (micro_statistics['k'] == k_value).astype(int)

# Generate dummy variables
dummies = pd.get_dummies(micro_statistics['x6'])
micro_statistics.rename(columns={'x6': 'cat_var'}, inplace=True)
dummies.columns = [f'x{i}' for i in range(6, 6 + dummies.shape[1])]
micro_statistics = pd.concat([micro_statistics, dummies], axis=1)


## 2. create the second set of micro moments (G3)
avg_prices = df.groupby('k')['Precio Unitario'].mean()



for k, price in avg_prices.items():
    micro_statistics.loc[micro_statistics['k'] == k, 'prices'] = price
    micro_statistics = pd.concat([micro_statistics, pd.DataFrame({'k': [k], 'prices': [price]})], ignore_index=True)

k
0    1.804686e+07
1    1.667907e+07
2    1.973762e+07
3    1.644347e+07
4    1.770722e+07
5    1.770260e+07
6    1.752879e+07
Name: Precio Unitario, dtype: float64


In [263]:
x_list = [f'x{i}' for i in range(6, 11)] # list of dummy variables for product categories 
print(x_list)

a_list = [f'a{i}' for i in range(1, len(unique_k))]
print(a_list)

['x6', 'x7', 'x8', 'x9', 'x10']
['a1', 'a2', 'a3', 'a4', 'a5', 'a6']


In [264]:
inside_micro_parts = {}
inside_micro_price = {} # dictionary to store the numerator of the price moments
micro_parts = {}

for num, j in enumerate(a_list, 1): 
    micro_parts[f'{j}=1'] = pyblp.MicroPart( #copying mid_part
        name = f'E[{j}=1]',
        dataset=micro_dataset,
        compute_values=lambda t, p, a: np.outer(a.demographics[:, num], np.r_[0, p.X2[:, 0]]), 
        #in mid_part they use X2, 
    )

    inside_micro_price[f'E[price*{j}=1]'] = pyblp.MicroPart( #copying inside_mid_part 
        name=f'E[{j}=1*price]',
        dataset=micro_dataset,
        compute_values=lambda t, p, a: np.outer(a.demographics[:, num], np.r_[1, p.X2[:, 3]]), 
        )
    
    for num2, i in enumerate(x_list, 1): 
        inside_micro_parts[f'E[{j}=1*{i}=1]'] = pyblp.MicroPart( #copying inside_mid_part 
            name=f'E[{j}=1*{i}=1]',
            dataset=micro_dataset,
            compute_values=lambda t, p, a: np.outer(a.demographics[:, num], np.r_[1, p.X2[:, num2]]), 
        )

In [ ]:
compute_ratio = lambda v: v[0] / v[1]
compute_ratio_gradient = lambda v: [1 / v[1], -v[0] / v[1]**2]

micro_moments = []

for num, j  in enumerate(a_list, 1): 
    value = micro_statistics.loc[ (micro_statistics[j] == 1), 'prices'].iloc[0]
    micro_moments.append(
        pyblp.MicroMoment(
            name=f'E[prices|{j}=1]',
            value = value,
            parts = [inside_micro_price[f'E[price*{j}=1]'], micro_parts[f'{j}=1']],
            compute_value=compute_ratio,
            compute_gradient=compute_ratio_gradient,
        )
    )
    
    for num2, i in enumerate(x_list, 1): 
        try: 
            value = micro_statistics.loc[(micro_statistics[j] == 1) & (micro_statistics[i] == 1), 'share'].iloc[0]
            micro_moments.append(
                pyblp.MicroMoment(
                    name=f'E[{i}=1|{j}=1]',
                    value=value, 
                    parts= [inside_micro_parts[f'E[{j}=1*{i}=1]'], micro_parts[f'{j}=1'] ], 
                    compute_value=compute_ratio,
                    compute_gradient=compute_ratio_gradient,
                )
            )
            print('works', j, i)
        except: 
            continue
        

### solving the problem

In [273]:
dim_X2 = 7

initial_sigma = np.eye(dim_X2) # square matrix with the dimension of X2 
initial_pi = np.array(np.ones((dim_X2, 1+len(unique_k)))) # dim(x2) x dim(agent formulation)


# We'll build the sigma_bounds so that off-diagonals are exactly 0
sigma_bounds = []
for i in range(dim_X2):
    for j in range(i + 1):
        if i == j:
            # Diagonal: allow > 0, no upper bound
            sigma_bounds.append((0.0, None))
        else:
            # Off-diagonal forced to 0
            sigma_bounds.append((0.0, 0.0))


# Build full k x k arrays for lb and ub
lb_sigma = np.zeros((dim_X2, dim_X2))
ub_sigma = np.zeros((dim_X2, dim_X2))

for i in range(dim_X2):
    for j in range(dim_X2):
        if i == j:
            # Diagonal: allow >= 0, no upper bound.
            lb_sigma[i, j] = 0.0
            # pyblp treats None as +∞
            ub_sigma[i, j] = None
        else:
            # Off-diagonal forced to 0
            lb_sigma[i, j] = 0.0
            ub_sigma[i, j] = 0.0

sigma_bounds = (lb_sigma, ub_sigma)


In [ ]:
results = problem.solve(
    sigma=initial_sigma,
    sigma_bounds  = sigma_bounds, 
    pi=initial_pi,
    optimization=pyblp.Optimization('bfgs', {'gtol': 1e-4}),
    iteration=pyblp.Iteration('squarem', {'atol': 1e-13}),
    #se_type='clustered',
    #W_type='clustered',
    micro_moments=micro_moments,
)
results